In [ ]:
# prompt: load my google drive
import pandas as pd
import numpy as np
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
outpatient_diagnosis_path = '/content/drive/MyDrive/Data/Output Data/outpatient_diagnosis.csv'
inpatient_diagnosis_path = '/content/drive/MyDrive/Data/Output Data/inpatient_diagnosis.csv'
professional_diagnosis_path = '/content/drive/MyDrive/Data/Output Data/professional_office_diagnosis.csv'

# HCUP CCSR LU Look Up file for ICD_CD
LU_CCSR_path = '/content/drive/MyDrive/Data/HealthCodes/DXCCSR_v2025-1_Concatenated.csv'

In [ ]:
# Reading diagnosis files from all settings and CCSR look up file
LU_CCSR = pd.read_csv(LU_CCSR_path)
outpatient_diagnosis = pd.read_csv(outpatient_diagnosis_path)
inpatient_diagnosis = pd.read_csv(inpatient_diagnosis_path)
professional_diagnosis = pd.read_csv(professional_diagnosis_path)


In [ ]:
inpatient_diagnosis

,BENE_ID,YR,ICD_DIAG_CD,ICD_Description
0,-10000010254618,2015,S134XX,Sprain of ligaments of cervical spine
1,-10000010254653,2015,Z3480,Encounter for supervision of other normal preg...
2,-10000010254653,2017,T7432X,Child psychological abuse
3,-10000010254656,2017,S8290X,Unspecified fracture of unspecified lower leg
4,-10000010254656,2018,Z3480,Encounter for supervision of other normal preg...
...,...,...,...,...
176759,-10000010287573,2021,J189,"Pneumonia, unspecified organism ..."
176760,-10000010287573,2022,K219,Gastro-esophageal reflux disease without esoph...
176761,-10000010287584,2016,O039,Complete or unspecified spontaneous abortion w...
176762,-10000010287673,2018,D649,"Anemia, unspecified ..."


In [ ]:
# prompt: for each diagnosis file, create a column called ICD_CD_TRUNC which should be the first three characters of ICD_DIAG_CD

def truncate_icd(df):
  df['ICD_CD_TRUNC'] = df['ICD_DIAG_CD'].astype(str).str[:3]
  return df

outpatient_diagnosis = truncate_icd(outpatient_diagnosis)
inpatient_diagnosis = truncate_icd(inpatient_diagnosis)
professional_diagnosis = truncate_icd(professional_diagnosis)


In [ ]:
outpatient_diagnosis

,BENE_ID,YR,ICD_DIAG_CD,ICD_Description,ICD_CD_TRUNC
0,-10000010254618,2015,R4689,Other symptoms and signs involving appearance ...,R46
1,-10000010254618,2016,R4689,Other symptoms and signs involving appearance ...,R46
2,-10000010254618,2016,B9789,Other viral agents as the cause of diseases cl...,B97
3,-10000010254618,2017,B9789,Other viral agents as the cause of diseases cl...,B97
4,-10000010254618,2018,B9789,Other viral agents as the cause of diseases cl...,B97
...,...,...,...,...,...
779180,-10000010287770,2017,Z558,Other problems related to education and literacy,Z55
779181,-10000010287770,2018,Z558,Other problems related to education and literacy,Z55
779182,-10000010287770,2020,Z558,Other problems related to education and literacy,Z55
779183,-10000010287949,2022,J329,"Chronic sinusitis, unspecified",J32


In [ ]:
LU_CCSR

,ICD_CD,CCSR_CATEGORY_CD,CCSR_CATEGORY_DESCRIPTION
0,A000,DIG001,Intestinal infection
1,A001,DIG001,Intestinal infection
2,A009,DIG001,Intestinal infection
3,A0100,DIG001,Intestinal infection
4,A0101,INF003,Bacterial infections
...,...,...,...
75233,Z9912,FAC012,Other specified encounters and counseling
75234,Z992,FAC025,Other specified status
75235,Z993,FAC025,Other specified status
75236,Z9981,FAC025,Other specified status


In [ ]:
# confirmed that all files BENE_ID, YR, ICD_DIAG_CD, ICD_Description
#outpatient_diagnosis.head(5)
#inpatient_diagnosis.head(5)
#professional_diagnosis.head(5)


In [ ]:
# prompt: create a dataframe with unique values of ICD_DIAG_CD from outpatient_diagnosis, inpatient_diagnosis, and professional_diagnosis.  Keep only the ICD_DIAG_CD

# Combine all ICD codes into a single Series
all_icd_codes = pd.concat([
    outpatient_diagnosis['ICD_DIAG_CD'],
    inpatient_diagnosis['ICD_DIAG_CD'],
    professional_diagnosis['ICD_DIAG_CD']
])

# Get unique ICD codes and create a DataFrame
unique_icd_df = pd.DataFrame({'ICD_DIAG_CD': all_icd_codes.unique()})

In [ ]:
unique_icd_df

,ICD_DIAG_CD
0,R4689
1,B9789
2,P292
3,I10
4,Z3480
...,...
381,P228
382,J8483
383,P229
384,P0730


In [ ]:
# prompt: merge LU_CCSR and unique_icd_df on ICD_CD=ICD_DIAG_CD.  Do the rows from unique_icd_df in the merged dataframe have non null values of CCSR_CATEGORY_DESCRIPTION?

# Merge LU_CCSR and unique_icd_df
merged_df = pd.merge(unique_icd_df, LU_CCSR, left_on='ICD_DIAG_CD', right_on='ICD_CD', how='left')

# Check for null values in CCSR_CATEGORY_DESCRIPTION for rows from unique_icd_df
null_ccsr_count = merged_df['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()
null_ccsr_count

50

In [ ]:
merged_df['CCSR_CATEGORY_DESCRIPTION'] = merged_df['CCSR_CATEGORY_DESCRIPTION'].fillna('UNK')
merged_df['CCSR_CATEGORY_CD'] = merged_df['CCSR_CATEGORY_CD'].fillna('UNK')


In [ ]:
# Count the frequency of each CCSR category description
merged_df['CCSR_CATEGORY_DESCRIPTION'].value_counts()
merged_df['CCSR_CATEGORY_CD'].value_counts()



,count
CCSR_CATEGORY_CD,
UNK,50
END003,21
FAC019,10
CIR011,10
MAL004,9
...,...
MBD002,1
MUS024,1
SKN005,1


In [ ]:
# prompt: drop ICD_CD and CCSR_CATEGORY_CD and called the new dataframe LU_CCSR_final

# Drop ICD_CD and CCSR_CATEGORY_CD columns
LU_CCSR_final = merged_df.drop(columns=['ICD_CD', 'CCSR_CATEGORY_CD'])
LU_CCSR_final



,ICD_DIAG_CD,CCSR_CATEGORY_DESCRIPTION
0,R4689,Symptoms of mental and substance use conditions
1,B9789,Viral infection
2,P292,Other specified and unspecified perinatal cond...
3,I10,Essential hypertension
4,Z3480,"Uncomplicated pregnancy, delivery or puerperium"
...,...,...
381,P228,Respiratory distress syndrome
382,J8483,Other specified and unspecified lower respirat...
383,P229,Respiratory distress syndrome
384,P0730,Short gestation; low birth weight; and fetal g...


In [ ]:
outpatient_diagnosis = pd.merge(outpatient_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
outpatient_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()



0

In [ ]:
inpatient_diagnosis = pd.merge(inpatient_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
inpatient_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

0

In [ ]:
professional_diagnosis = pd.merge(professional_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
professional_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

0

In [ ]:
# prompt: from each of the three files in the previous cells, drop ICD_DIAG_CD and ICD_Description and keep only unique rows

# Drop columns and keep unique rows for each DataFrame
outpatient_categories = outpatient_diagnosis.drop(columns=['ICD_DIAG_CD', 'ICD_Description']).drop_duplicates()
inpatient_categories = inpatient_diagnosis.drop(columns=['ICD_DIAG_CD', 'ICD_Description']).drop_duplicates()
professional_categories = professional_diagnosis.drop(columns=['ICD_DIAG_CD', 'ICD_Description']).drop_duplicates()


In [ ]:
# prompt: for each category file, create a flag.  For example, INP in inpatient_categories should be Y.

# Create flags for each category
outpatient_categories['OP_FLAG'] = 'Y'
inpatient_categories['IP_FLAG'] = 'Y'
professional_categories['PROF_FLAG'] = 'Y'


In [ ]:
# Merge the three dataframes
master_categories = pd.merge(outpatient_categories, inpatient_categories,
                             on=['BENE_ID', 'YR', 'ICD_CD_TRUNC','CCSR_CATEGORY_DESCRIPTION'],
                             how='outer')
master_categories['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()




0

In [ ]:
master_categories = pd.merge(master_categories, professional_categories,
                             on=['BENE_ID', 'YR', 'ICD_CD_TRUNC', 'CCSR_CATEGORY_DESCRIPTION'],
                             how='outer')
master_categories['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()


0

In [ ]:
# prompt: replace null values of OP_FLAG, IP_FLAG, PROF_FLAG with 'N' with a technique that is compatible in pandas 3.0

# Replace null values in 'OP_FLAG', 'IP_FLAG', and 'PROF_FLAG' with 'N'
master_categories['OP_FLAG'] = master_categories['OP_FLAG'].fillna('N')
master_categories['IP_FLAG'] = master_categories['IP_FLAG'].fillna('N')
master_categories['PROF_FLAG'] = master_categories['PROF_FLAG'].fillna('N')


In [ ]:
# prompt: count the number of rows by CCSR_CATEGORY_DESCRIPTION and ICD_CD_TRUC
row_counts_CCSR = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION']).size().reset_index(name='counts')
row_counts_ICD_TRUCN = master_categories.groupby(['ICD_CD_TRUNC']).size().reset_index(name='counts')
row_counts_both = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION', 'ICD_CD_TRUNC']).size().reset_index(name='counts')



In [ ]:
row_counts_CCSR

,CCSR_CATEGORY_DESCRIPTION,counts
0,Abdominal pain and other digestive/abdomen sig...,324
1,Abnormal findings without diagnosis,25998
2,Acquired absence of limb or organ,3602
3,Acute and chronic tonsillitis,2120
4,Acute bronchitis,4270
...,...,...
134,Trauma- and stressor-related disorders,11
135,UNK,49199
136,"Uncomplicated pregnancy, delivery or puerperium",2056
137,Urinary tract infections,1639


In [ ]:
row_counts_ICD_TRUCN

,ICD_CD_TRUNC,counts
0,0FB,3
1,0FC,1
2,A41,1899
3,A49,7
4,B00,2966
...,...,...
229,Z90,3402
230,Z91,6105
231,Z94,5163
232,Z95,15616


In [ ]:
row_counts_both

,CCSR_CATEGORY_DESCRIPTION,ICD_CD_TRUNC,counts
0,Abdominal pain and other digestive/abdomen sig...,R13,194
1,Abdominal pain and other digestive/abdomen sig...,R19,130
2,Abnormal findings without diagnosis,R73,3766
3,Abnormal findings without diagnosis,R76,17
4,Abnormal findings without diagnosis,R93,22215
...,...,...,...
242,Urinary tract infections,N30,683
243,Urinary tract infections,N39,529
244,Viral infection,B08,1786
245,Viral infection,B34,3508


In [ ]:
# prompt: filter row_counts_both for CCSR_CATEGORY_DESCRIPTION == 'UNK'

row_counts_both[row_counts_both['CCSR_CATEGORY_DESCRIPTION'] == 'UNK']


,CCSR_CATEGORY_DESCRIPTION,ICD_CD_TRUNC,counts
209,UNK,0FB,3
210,UNK,0FC,1
211,UNK,F02,226
212,UNK,L89,30
213,UNK,M80,372
214,UNK,S01,192
215,UNK,S06,1892
216,UNK,S13,284
217,UNK,S22,258
218,UNK,S26,1


In [ ]:
# prompt: if ICD_CD_TRUNC in master_categories is in the following range S00-T88  and CCSR_CATEGORY_DESCRIPTION is UNK then recode CCSR_CATEGORY_DESCRIPTION to equal 'Injury or  poisoning and others'

# Create a boolean mask for ICD codes within the specified range and CCSR_CATEGORY_DESCRIPTION equal to 'UNK'
mask = (master_categories['ICD_CD_TRUNC'].between('S00', 'T88', inclusive='both')) & (master_categories['CCSR_CATEGORY_DESCRIPTION'] == 'UNK')

# Recode CCSR_CATEGORY_DESCRIPTION for the selected rows
master_categories.loc[mask, 'CCSR_CATEGORY_DESCRIPTION'] = 'Injury or poisoning and others'


In [ ]:
row_counts_CCSR = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION']).size().reset_index(name='counts')
row_counts_ICD_TRUCN = master_categories.groupby(['ICD_CD_TRUNC']).size().reset_index(name='counts')
row_counts_both = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION', 'ICD_CD_TRUNC']).size().reset_index(name='counts')



In [ ]:
row_counts_CCSR

,CCSR_CATEGORY_DESCRIPTION,counts
0,Abdominal pain and other digestive/abdomen sig...,324
1,Abnormal findings without diagnosis,25998
2,Acquired absence of limb or organ,3602
3,Acute and chronic tonsillitis,2120
4,Acute bronchitis,4270
...,...,...
135,Trauma- and stressor-related disorders,11
136,UNK,635
137,"Uncomplicated pregnancy, delivery or puerperium",2056
138,Urinary tract infections,1639


In [ ]:
master_categories.to_csv('/content/drive/MyDrive/Data/Output Data/master_categories.csv', index=False)
